# Over-fitting을 방지하기 위해서 한 일들  
- SegNet을 CamVid data set을 가지고 구현하면서 겪은 골치 아픈 일이 over fitting 현상이다  
- over fitting을 방지하기 위해 내가 어떤 코드를 작성했고 어떤 판단을 내렸는지 기록한다

- 오버피팅을 방지할 수 있는 방법들은 다음과 같다    
    - 1. Batch Normalization  
    - 2. Drop out  
    - 3. weight initialize  
    - 4. 모델의 레이어를 조금 더 간단하게 구성  
    - 5. Early Stopping  
    - 6. Image Data Augmentation  
    - 7. Train data set 추가
    - 이 외에도 다양한 방법들이 존재한다

### 1. Add Batch Normalization Layer  
- Batch Norm이란?  
    - 모델을 훈련할 때 메모리를 효율적으로 사용하기 위해 이미지를 batch 별로 나눠서 훈련을 한다  
    - batch norm이란 이러한 batch의 평균과 분산을 이용해 정규화를 시키는 방법이다  
    - 다른 정규화 기법으로 instance norm, layer norm, group norm 등이 있다
- 이는 이미 논문에서도 Convolution Block 구성에서 Conv + BN + ReLU로 되어있고  
- SegNet을 구현하는 과정에서도 BN을 사용했으므로 pass

### 2. Add Drop Out Layer  
- Drop out이란?  
    - Drop out은 레이어에서 다음 레이어로 weight를 0으로 전달하는 것이다  
    - 즉, 다음 레이어에 영향을 주지 않도록 하는 것이다
- Drop out layer를 적용해 성능이 어떤지 실험해보기 전에  
- Drop out layer를 추가하는 것에 대한 나의 생각을 적어본다  
    - 개인적으로 생각하기엔 Segmentation task에서 model을 구성할 때 drop out layer를 적용하는 것은 큰 리스크가 있다고 생각한다  
    - Segmentation에서는 이미지의 pixel 하나 하나가 모두 중요한 정보라고 생각한다  
    - 특히나 CamVid data set의 경우 object가 작은 pedestrian이나 bicyclist같은 경우 segmentation을 수행하지 못하면 이 모델을 자율 주행의 목적으로 사용한다고 한다면 굉장히 위험한 일이다  
    - 그래서 이러한 상황에서 drop out layer를 추가한다면 중요한 feature의 weight를 전달하지 못하는 일이 생길 수도 있다고 생각한다  

In [ ]:
# define encoder block module using pre-trained vgg16
class EncoderBlock(nn.Module):
    def __init__(self, index, dropout=True):
        super(EncoderBlock, self).__init__()
        # build encoder network (Conv + BN + ReLU)
        block = [
            vgg16.features[index],
            nn.BatchNorm2d(vgg16.features[index].out_channels),
            nn.ReLU(inplace=True)
        ]
        if dropout:
            block.append(nn.Dropout(0.2))
        self.block = nn.Sequential(*block)
                
    def forward(self, x):
        return self.block(x)
    
# define encoder block module
class DecoderBlock(nn.Module):
    def __init__(self, in_dim, out_dim, dropout=True):
        super(DecoderBlock, self).__init__()
        # build decoder network (Conv + BN + ReLU)
        block = [
            nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_dim),
            nn.ReLU(inplace=True)
        ]
        if dropout:
            block.append(nn.Dropout(0.2))
        self.block = nn.Sequential(*block)
        
    def forward(self, x):
        return self.block(x)

**Result**
- Drop out layer를 추가해봤다  
- 이때 Encoder Network의 첫 번째 Block과 Decoder의 마지막 Block에는 drop out을 적용하지 않았다  
- 적어도 input으로 들어오는 이미지의 모든 pixel은 feature로 전달 받아야 Segmentation을 잘 수행한다고 판단했다  
- 결과적으로 보면 Segmentation을 잘 수행하지 못했다  
- 그래서 Drop out layer를 적용하는 것은 X

### 3. weight initialization  
- 딥러닝 학습에서는 가중치가 매우 중요하다  
- 가중치 하나로 인해 모델의 결과가 완전히 달라질 수 있다  
- 그래서 특정 분포나 Xavier, Kaiming, LeCun 등 다른 초기화 방법을 따라 가중치를 초기화 시켜 학습에 안정이 되게끔 해준다

In [ ]:
# define encoder block module using pre-trained vgg16
class EncoderBlock(nn.Module):
    def __init__(self, index):
        super(EncoderBlock, self).__init__()
        # build encoder network (Conv + BN + ReLU)
        self.block = nn.Sequential(
            vgg16.features[index],
            nn.BatchNorm2d(vgg16.features[index].out_channels),
            nn.ReLU(inplace=True),
        )
        self._initialize_weights_()                    
        
    # he normal initialization
    def _initialize_weights_(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data)
                
    def forward(self, x):
        return self.block(x)
    
# define encoder block module
class DecoderBlock(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(DecoderBlock, self).__init__()
        # build decoder network (Conv + BN + ReLU)
        self.block = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_dim),
            nn.ReLU(inplace=True),
        )
        self._initialize_weights_()                    
        
    # he normal initialization
    def _initialize_weights_(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data)
        
    def forward(self, x):
        return self.block(x)

**Result**  
- weight들을 초기화하나 안하나 output은 별 차이가 없는 것 같다

### 4. 모델의 레이어를 조금 더 간단하게 구성  
- 내가 구축한 SegNet은 5개의 Encoder Network, 이와 대칭인 5개의 Decoder Network로 이루어져 있다  
- 조금 더 간단하게 구성해 4개의 Encoder와 Decoder 혹은 3개로 구성했다  

**Result**  
- 4개의 Encoder Network, 이와 대칭인 4개의 Decoder Network  
- 그리고 3개의 Encoder Network, 이와 대칭인 3개의 Decoder Network 두 가지 구성으로 실험을 해 본 결과  
- 5개의 네트워크로 구성한 것과는 별 차이가 없다

### 5. Early Stopping  
- early stopping은 over-fitting을 방지할 수 있는 가장 편리하면서도 쉬운 방법이라 생각한다  
- patience의 value를 지정해 해당 value만큼 validation loss가 개선되지 않을 경우 학습을 종료시켜버리는 방식이다  
- Train data가 학습되기도 전에 validation loss가 증가해서 early stopping되는 경우가 많았다  
- patience는 현재 20으로 설정했지만 40정도로 더 크게 잡아야 하나 생각한다  
- 어떻게 해결해야 할까,,,  

- 사실 early stopping은 오버피팅이 되지 않도록 강제로 종료하는 것이지  
- 오버피팅 문제를 해결해주는건 아니다

In [ ]:
# We set early stopping to check over fitting of model
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.3f} --> {val_loss:.3f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

### 6. Data Augmentation  
- Data Augmentation은 이미지 데이터를 좌우 반전, 상하 반전, 회전 등 다양한 변환을 적용해 이미지 데이터를 증강시키는 기법이다  
- 개인적인 생각으로는 Segmentation task에는 data augmentation이 적절치 않다고 생각한다  
- 이미지 pixel 하나 하나를 class 별로 분류해야 하는 작업인데 augmentation을 통해 모델을 훈련시키는 과정에서 weight들이 잘못 훈련될 수 있기 때문이다  

In [ ]:
transforms_ = [
    transforms.ToTensor(),
    transforms.RandomVerticalFlip(p=1),
    transforms.RandomAffine(30),
    transforms.RandomPerspective(),
    transforms.RandomRotation(30, expand=False),
]

**Result**
- Data Augmentation의 결과 역시 좋지 않다  
- 따라서 Segmentation task에는 Augmentation이 좋은 방법은 아니다

### 7. Train data 추가  
- 현재 쓰고 있는 CamVid data set은 kaggle에서 얻어온 데이터다  
- 이는 train은 367장, valid는 101장, test는 233장으로 구성되어 있다  
- 이 데이터들을 각각 train, valid, test 그대로 사용을 했으나  
- valid loss가 0.5에서 줄어들지 않고 mIoU score는 0.4에서 증가하지 않았다  
- 그래서 이 3가지 data set을 합쳐 shuffling한 후  
- 다시 train은 500장, valid는 100장, test는 101장으로 사용해본 결과  

In [ ]:
# data 합치기 (총 701장)
data_images = np.concatenate((train_image,valid_image,test_image), axis=0)
data_labels = np.concatenate((train_label,valid_label,test_label), axis=0)

# shuffling
idx = np.arange(len(data_images))
np.random.shuffle(idx)
shuffled_images = data_images[idx]
shuffled_labels = data_labels[idx]

# divide train, valid and test data set
# train data: 500장
train_image = shuffled_images[:500]
train_label = shuffled_labels[:500]
# valid data: 100장
valid_image = shuffled_images[500:600]
valid_label = shuffled_labels[500:600]
# test data: 101장
test_image = shuffled_images[600:]
test_label = shuffled_labels[600:]

**Result**  
- valid loss가 가끔 0.3 후반대가 나올 때도 있고 valid mIoU도 0.5 이상으로 증가한 것을 확인했다  
- 아무래도 train data set을 조금 더 많은 데이터로 훈련시켜야 학습이 잘 되는 듯 하다